In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    #print("nse3 : ", nse3)
    #print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'nak'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:

#run_num = range(len(folder))

run_num = [0, 1]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/nak/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['도개_2013.xlsx', '도개_2014.xlsx', '도개_2015.xlsx', '도개_2016.xlsx', '도개_2017.xlsx', '도개_2018.xlsx', '도개_2019.xlsx', '도개_2020.xlsx'], ['신암_2013.xlsx', '신암_2014.xlsx', '신암_2015.xlsx', '신암_2016.xlsx', '신암_2017.xlsx', '신암_2018.xlsx', '신암_2019.xlsx', '신암_2020.xlsx'], ['풍양_2013.xlsx', '풍양_2014.xlsx', '풍양_2015.xlsx', '풍양_2016.xlsx', '풍양_2017.xlsx', '풍양_2018.xlsx', '풍양_2019.xlsx', '풍양_2020.xlsx']]
data/nak/자동/도개_2013.xlsx
data/nak/자동/도개_2014.xlsx
data/nak/자동/도개_2015.xlsx
data/nak/자동/도개_2016.xlsx
data/nak/자동/도개_2017.xlsx
data/nak/자동/도개_2018.xlsx
data/nak/자동/도개_2019.xlsx
data/nak/자동/도개_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-7-13 23:00
data/nak/자동/신암_2013.xlsx
data/nak/자동/신암_2014.xlsx
data/nak/자동/신암_2015.xlsx
data/nak/자동/신암_2016.xlsx
data/nak/자동/신암_2017.xlsx
data/nak/자동/신암_2018.xlsx
data/nak/자동/신암_2019.xlsx
data/nak/자동/신암_2020.xlsx
time range in files :  2013-01-01 

1/1 [==============================] - 0s 21ms/step - gen_loss: 10.2186 - disc_loss: 0.3294 - rmse: 0.4109 - val_loss: 0.3517
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.8351 - disc_loss: 0.3363 - rmse: 0.3840 - val_loss: 0.4302
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.0188 - disc_loss: 0.3440 - rmse: 0.3796 - val_loss: 0.4204
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.1965 - disc_loss: 0.3429 - rmse: 0.3794 - val_loss: 0.3641
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.5020 - disc_loss: 0.3451 - rmse: 0.3884 - val_loss: 0.4560
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.2928 - disc_loss: 0.3339 - rmse: 0.4120 - val_loss: 0.4839
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.4159 - disc_loss: 0.3176 - rmse: 0.3787 - val_loss: 0.3650
Epoch 56/2000
1/1 [=================

Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8336 - disc_loss: 0.2528 - rmse: 0.3061 - val_loss: 0.4021
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3914 - disc_loss: 0.2628 - rmse: 0.3645 - val_loss: 0.2593
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0377 - disc_loss: 0.2588 - rmse: 0.3372 - val_loss: 0.3023
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9754 - disc_loss: 0.2616 - rmse: 0.2783 - val_loss: 0.3052
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.7765 - disc_loss: 0.2632 - rmse: 0.3191 - val_loss: 0.3823
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0814 - disc_loss: 0.2533 - rmse: 0.3049 - val_loss: 0.3952
Epoch 114/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.5924 - disc_loss: 0.2633 - rmse: 0.3418 - val_loss: 0.3042
Epoch 115/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3776 - disc_loss: 0.2577 - rmse: 0.3088 - val_loss: 0.2788
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6800 - disc_loss: 0.2456 - rmse: 0.2891 - val_loss: 0.2664
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7626 - disc_loss: 0.2467 - rmse: 0.3063 - val_loss: 0.3235
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3450 - disc_loss: 0.2513 - rmse: 0.3138 - val_loss: 0.3661
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1526 - disc_loss: 0.2511 - rmse: 0.2625 - val_loss: 0.3255
Epoch 171/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 4.9169 - disc_loss: 0.2579 - rmse: 0.2694 - val_loss: 0.3487
Epoch 172/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 6.4298 - disc_loss: 0.2440 - rmse: 0.3615 - val_loss: 0.3204
Epoch 173/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1961 - disc_loss: 0.2287 - rmse: 0.3323 - val_loss: 0.2813
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7379 - disc_loss: 0.2592 - rmse: 0.3197 - val_loss: 0.3358
Epoch 226/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.6021 - disc_loss: 0.2412 - rmse: 0.2694 - val_loss: 0.3620
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8189 - disc_loss: 0.2434 - rmse: 0.3280 - val_loss: 0.2675
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7877 - disc_loss: 0.2387 - rmse: 0.2685 - val_loss: 0.2961
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0611 - disc_loss: 0.2394 - rmse: 0.3086 - val_loss: 0.3169
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8388 - disc_loss: 0.2225 - rmse: 0.2926 - val_loss: 0.3415
Epoch 231/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8997 - disc_loss: 0.1866 - rmse: 0.2476 - val_loss: 0.2292
Epoch 921/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5193 - disc_loss: 0.1732 - rmse: 0.2535 - val_loss: 0.2550
Epoch 922/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9568 - disc_loss: 0.1844 - rmse: 0.2492 - val_loss: 0.3771
Epoch 923/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2783 - disc_loss: 0.1891 - rmse: 0.2407 - val_loss: 0.3096
Epoch 924/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7491 - disc_loss: 0.1798 - rmse: 0.2557 - val_loss: 0.2622
Epoch 925/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7174 - disc_loss: 0.1805 - rmse: 0.1966 - val_loss: 0.3313
Epoch 926/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2312 - disc_loss: 0.1741 - rmse: 0.2883 - val_loss: 0.2753
Epoch 927/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1634 - disc_loss: 0.1708 - rmse: 0.2101 - val_loss: 0.2584
Epoch 979/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0474 - disc_loss: 0.1679 - rmse: 0.2615 - val_loss: 0.2606
Epoch 980/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4772 - disc_loss: 0.1661 - rmse: 0.2571 - val_loss: 0.2282
Epoch 981/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9602 - disc_loss: 0.1879 - rmse: 0.2408 - val_loss: 0.2230
Epoch 982/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3011 - disc_loss: 0.1757 - rmse: 0.2453 - val_loss: 0.2678
Epoch 983/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1801 - disc_loss: 0.1771 - rmse: 0.2502 - val_loss: 0.4012
Epoch 984/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2254 - disc_loss: 0.1835 - rmse: 0.2478 - val_loss: 0.2500
Epoch 985/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3820 - disc_loss: 0.1657 - rmse: 0.2711 - val_loss: 0.2513
Epoch 1037/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9068 - disc_loss: 0.1796 - rmse: 0.3355 - val_loss: 0.2623
Epoch 1038/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4208 - disc_loss: 0.1762 - rmse: 0.2351 - val_loss: 0.2538
Epoch 1039/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9041 - disc_loss: 0.1688 - rmse: 0.3218 - val_loss: 0.3808
Epoch 1040/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1548 - disc_loss: 0.1787 - rmse: 0.2449 - val_loss: 0.2547
Epoch 1041/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1140 - disc_loss: 0.1772 - rmse: 0.2690 - val_loss: 0.2375
Epoch 1042/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5026 - disc_loss: 0.1651 - rmse: 0.1903 - val_loss: 0.2718
Epoch 1043/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6835 - disc_loss: 0.1758 - rmse: 0.2353 - val_loss: 0.2904
Epoch 1095/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7817 - disc_loss: 0.1769 - rmse: 0.2506 - val_loss: 0.3135
Epoch 1096/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9937 - disc_loss: 0.1722 - rmse: 0.2287 - val_loss: 0.2617
Epoch 1097/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8765 - disc_loss: 0.1585 - rmse: 0.2278 - val_loss: 0.2658
Epoch 1098/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5955 - disc_loss: 0.1785 - rmse: 0.2794 - val_loss: 0.3238
Epoch 1099/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0166 - disc_loss: 0.1854 - rmse: 0.2259 - val_loss: 0.2905
Epoch 1100/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3576 - disc_loss: 0.1805 - rmse: 0.2360 - val_loss: 0.2636
Epoch 1101/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3512 - disc_loss: 0.1862 - rmse: 0.2510 - val_loss: 0.3699
MissData :  save/  miss :  (10650, 12)
1/1 [==============================] - 0s 4ms/step - loss: 0.2405
MissData :  save/  miss :  (10650, 12)
interpol flag :  [True, False]
folder :  data/nak/총량/
colum_idx :  :,[28,30,31,36,40,43,44]
file_names[idx] :  [['상주3_2013.xlsx', '상주3_2014.xlsx', '상주3_2015.xlsx', '상주3_2016.xlsx', '상주3_2017.xlsx', '상주3_2018.xlsx', '상주3_2019.xlsx', '상주3_2020.xlsx'], ['위천6_2013.xlsx', '위천6_2014.xlsx', '위천6_2015.xlsx', '위천6_2016.xlsx', '위천6_2017.xlsx', '위천6_2018.xlsx', '위천6_2019.xlsx', '위천6_2020.xlsx'], ['병성천-1_2013.xlsx', '병성천-1_2014.xlsx', '병성천-1_2015.xlsx', '병성천-1_2016.xlsx', '병성천-1_2017.xlsx', '병성천-1_2018.xlsx', '병성천-1_2019.xlsx', '병성천-1_2020.xlsx']]
data/nak/총량/상주3_2013.xlsx
data/nak/총량/상주3_2014.xlsx
data/nak/총량/상주3_2015.xlsx
data/nak/총량/상주3_2016.xlsx
data/nak/총량/상주3_2017.xlsx
data/nak/총량/상주3_2018.xlsx
data/nak/총량/상주3_2019.xlsx
data/na

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [17]:
print(real_df_all.shape)
rnn_target_column = 'tp'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(66024, 69)
tp


In [18]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (46216, 69) val_df.shape :  (13205, 69) test_df.shape: (6603, 69)


In [19]:
#fake_df.shape[0]/8760

In [20]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [21]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  6
out_num_features :  1


In [22]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [23]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/nak/models/nak/phosphorus/


In [26]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [27]:
rnn_epochs, rnn_in_setps

(20, 240)

In [28]:
'''
sub_flag = True
history_nse = []

bob = 0
b = 3

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15 + bob,
        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    nse, _, _, _ = compa(
        model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -7):
        b = 3
    if(bob > 7):
        b = -3
'''
    


'\nsub_flag = True\nhistory_nse = []\n\nbob = 0\nb = 3\n\nwhile sub_flag:\n    \n\n    \n    gru_model = model_gru(\n        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,\n        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15 + bob,\n        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)\n    nse, _, _, _ = compa(\n        model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,\n        #predict_day = rnn_predict_day)\n        predict_day = 4)\n    \n    if(nse > 0.5):\n        sub_flag = False\n   \n    \n    history_nse.append(nse)\n    \n    print(\'--------------------------\')\n    print(history_nse)\n    print(nse)\n    print(\'--------------------------\')\n    \n        \n    bob += b\n    if(bob < -7):\n        b = 3\n    if(bob > 7):\n        b = -3\n'

## 모델 학습

In [30]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 10,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

Epoch 1/15
10/10 [==============================] - 17s 2s/step - loss: 0.7962 - mean_absolute_error: 0.5922 - nse: 0.2001 - val_loss: 0.3206 - val_mean_absolute_error: 0.4058 - val_nse: 0.3840
Epoch 2/15
10/10 [==============================] - 17s 2s/step - loss: 0.4386 - mean_absolute_error: 0.4453 - nse: 0.5345 - val_loss: 0.2163 - val_mean_absolute_error: 0.3148 - val_nse: 0.5912
Epoch 3/15
10/10 [==============================] - 17s 2s/step - loss: 0.3424 - mean_absolute_error: 0.3871 - nse: 0.6075 - val_loss: 0.2043 - val_mean_absolute_error: 0.3050 - val_nse: 0.6146
Epoch 4/15
10/10 [==============================] - 17s 2s/step - loss: 0.3298 - mean_absolute_error: 0.3734 - nse: 0.6211 - val_loss: 0.2198 - val_mean_absolute_error: 0.3298 - val_nse: 0.5823
Epoch 5/15
10/10 [==============================] - 17s 2s/step - loss: 0.3131 - mean_absolute_error: 0.3669 - nse: 0.6130 - val_loss: 0.2112 - val_mean_absolute_error: 0.3102 - val_nse: 0.6013
Epoch 6/15
10/10 [============

## core / window.py / 

In [31]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print('nse_val : ', val_nse['GRU'])
#print("---------------")
#print(val_nse['GRU'])
#print(val_pbias['GRU'])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.017709987469433783
predict_day :  5
------------------------
0.004983532661541205 6.208650467279969e-05 0.007879499011536183
save/nak/models/nak/phosphorus/
year : 2013 ~ 2020
run :  [0, 1]
target :  tp   6
length :  66024
train=7, test=1, val=2
nse_val :  [0.53664018]


In [32]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print('nse_all : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.019415773778834786
predict_day :  5
------------------------
0.005278292378155459 0.00014325287333616222 0.011968829238324116
nse_all :  [0.5249636]


In [33]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.01889937682272426
predict_day :  5
------------------------
0.004568814988366927 5.875034708490405e-05 0.00766487749966717
nse_train :  [0.72936127]


In [34]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_test : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.026990148149948722
predict_day :  5
------------------------
0.010326233965504191 0.0009605369969584268 0.030992531309307843
nse_test :  [0.10430199]


In [ ]:
#train_df.shape, val_df.shape, test_df.shape

In [ ]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column, " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
